# Úvodné nastavenie

## Načítanie knižníc

In [ ]:
%load_ext autoreload

In [ ]:
from pathlib import Path
import pandas as pd 
import numpy as np
from datetime import datetime
import random
import os
import re




In [ ]:
# Aby fungovalo nacitanie spolocnych funkcii, je nutne ich nainstalovat cez `pip install .` na urovni, kde sa nachadza pyproject.toml

from data_loading import load_data_zp, load_zoznam_obci, load_psc_2_zuj, load_pzs_12
from OSN_common.helpers import norm_path, standardize_text
from OSN_common.constants import COLUMNS_274, COLUMNS_VS

%autoreload 2

# from OSN_common.nacitanie_dat import nacitaj_prevodovy_subor, nacitaj_zoznam_obci

## Konfigurácia vstupných a výstupných súborov

In [ ]:
_paths = {
    "vystupy": Path('.') / "Vystupy"
}

_datum = datetime.now().strftime("%Y%m%d_%H%M%S")

_paths['vystupy'] = _paths['vystupy'] / _datum
_paths['vystupy'].mkdir()

_paths['validacia'] = _paths['vystupy'] / 'validacia'
_paths['validacia'].mkdir()

print(f'Ukladam data do priecinku {_paths["vystupy"]}')

data_folder = norm_path(Path(os.environ.get('OSN_data')) / "11_Dátové súbory a prevodníky" / "06_Dáta od zdravotných poisťovní")

Ukladam data do priecinku Vystupy/20250505_133302


## Nastavenie roku a zoznamu poisťovní

In [ ]:
_rok = "2024"
_spracovat = ["24", "27"]

## Nastavenie validácií

In [ ]:
VALIDATION_COLUMNS = ['kod_zp', 'pzs_6', 'id_hp', 'id_hp_pzs', 'id_poistenca', 'nazov_validacie', 'ovplyvnene_tabulky', 'zavaznost', 'odovodnenie']
VALIDATION_DEFINITIONS = pd.read_excel(data_folder / 'Validácie' / 'Validacie-rocnej-davky-2024-OSN.xlsx', sheet_name='validacie', index_col=0).to_dict(orient='index')

VALIDATIONS = {kod_zp: pd.DataFrame(columns=VALIDATION_COLUMNS) for kod_zp in _spracovat}

In [ ]:
def format_with_row_columns(text: str) -> str:
    """Convert {column_name} in text to {row[column_name]} for DataFrame row access.
    
    e.g. "ID poistenca: {id_poistenca}" -> "ID poistenca: {row[id_poistenca]}"
    """
    pattern = r'\{([a-zA-Z_][a-zA-Z0-9_]*)\}'
    return re.sub(pattern, lambda m: f"{{row[{m.group(1)}]}}", text)

def add_validation_columns(df: pd.DataFrame, nazov_validacie: str, ovplyvnene_tabulky: str, zavaznost: int, odovodnenie: str) -> pd.DataFrame:
    """Add validation columns to the DataFrame."""
    odovodnenia = df.apply(lambda row: format_with_row_columns(odovodnenie).format(row=row), axis=1)
    out = df.filter(VALIDATION_COLUMNS).copy()
    out['nazov_validacie'] = nazov_validacie
    out['ovplyvnene_tabulky'] = ovplyvnene_tabulky
    out['zavaznost'] = zavaznost
    out['odovodnenie'] = odovodnenia
    return out

def register_validation(invalid_df: pd.DataFrame, id_validacie: str):
    """Register the validation in the validation dictionary."""
    invalid = add_validation_columns(invalid_df, **VALIDATION_DEFINITIONS[id_validacie])

    for kod_zp in _spracovat:
        if kod_zp in invalid['kod_zp'].values:
            VALIDATIONS[kod_zp] = pd.concat([VALIDATIONS[kod_zp], invalid[invalid['kod_zp']==kod_zp][VALIDATION_COLUMNS]], ignore_index=True)

# Načítanie dát

In [ ]:
_data = load_data_zp(data_folder, _rok, _spracovat)


Nacitavam data pre poistovnu dovera
Nacitavam data zo suboru 2024_24_01_UZS_JZS.csv
Nacitavam data zo suboru 2024_24_02_HP.csv


Nacitavam data zo suboru 2024_24_03_PREKLAD.csv
Nacitavam data zo suboru 2024_24_04_VDG.csv
Nacitavam data zo suboru 2024_24_05_VYKON.csv
Nacitavam data zo suboru 2024_24_06_PPDRG.csv
Nacitavam data zo suboru 2024_24_09_POISTENCI.csv
Nacitavam data zo suboru 2024_24_14_SUMAR.csv
Nacitavam data pre poistovnu union
Nacitavam data zo suboru 2024_27_01_UZS_JZS.csv
Nacitavam data zo suboru 2024_27_02_HP.csv
Nacitavam data zo suboru 2024_27_03_PREKLAD.csv
Nacitavam data zo suboru 2024_27_04_VDG.csv
Nacitavam data zo suboru 2024_27_05_VYKON.csv
Nacitavam data zo suboru 2024_27_06_PPDRG.csv
Nacitavam data zo suboru 2024_27_09_POISTENCI.csv
Nacitavam data zo suboru 2024_27_14_SUMAR.csv


# Predspracovanie dát

## Priprav dat UZS_JZS (`uzs`)

### Predspracuj data zo suboru 01_UZS_JZS

In [ ]:
# Najdi relevantne tabuľky
_uzs = pd.concat([_data[k]['uzs_jzs'] for k in _spracovat], ignore_index=True)

len(_uzs)

493777

In [ ]:
# Vyluc riadky, kde typ hospitalizacie je "pripocitatelna polozka"
_uzs = _uzs[_uzs['typ_hospitalizacie'] != 'Z']

len(_uzs)

493777

In [ ]:
_uzs.columns

Index(['kod_zp', 'id_poistenca', 'obdobie', 'pzs_12', 'id_hp', 'novorodenec',
       'datum_od', 'datum_do', 'pohyb_poistenca', 'dgn_prijem',
       'dgn_prepustenie', 'kod_jednodnoveho_vykonu', 'kod_hlavneho_vykonu',
       'typ_hospitalizacie', 'pzs_12_odosielatel', 'datum_ziadanky',
       'typ_starostlivosti', 'id_navrhu_planovana', 'uhrada_hp', 'uhrada_pp',
       'uhrada_imzs', 'uhrada_emzs', 'kod_ms', 'uroven_ms', 'kod_programu'],
      dtype='object')

### VALIDATION id_hp_chybajuce_01


In [ ]:
# Prirad cislo riadku ako ID_HP pre chybajuce ID
# Takato hospitalizacia sa neda naparovat na ostatne subory (nie je problem v pripade JZS), a pri ostatnych je to lepsie ako ich vyhodit
if _uzs.id_hp.isna().any():
    _uzs.loc[_uzs.id_hp.isna(), 'id_hp'] = 'x' + _uzs[_uzs.id_hp.isna()].index.map(str)

In [ ]:
# Exportuj riadky, kde chybalo ID hospitalizacie a typ je DRG alebo UH.

id_validacie = 'id_hp_chybajuce_01'

invalid = _uzs[_uzs['id_hp'].str.startswith('x') & _uzs['typ_starostlivosti'].isin(['DRG', 'UH'])].copy()
invalid['pzs_6'] = invalid['pzs_12'].str[:6]
invalid['id_hp_pzs'] = pd.NA

register_validation(invalid, id_validacie)


In [ ]:
# Vypíš záporné úhrady
UHRADY_COLUMNS = ['uhrada_hp', 'uhrada_pp', 'uhrada_imzs', 'uhrada_emzs']

id_validacie = 'uhrada_zaporna_01'

invalid = _uzs[_uzs[UHRADY_COLUMNS].lt(0).any(axis=1)].copy()
invalid['pzs_6'] = invalid['pzs_12'].str[:6]
invalid['id_hp_pzs'] = pd.NA

register_validation(invalid, id_validacie)

In [ ]:
# Ocisti kody vykonov a diagnoz na standardizovany tvar - len male pismena a cisla.

_stlpce_s_kodmi = ['kod_hlavneho_vykonu', 'kod_jednodnoveho_vykonu', 'dgn_prijem', 'dgn_prepustenie']

_uzs[_stlpce_s_kodmi] = _uzs[_stlpce_s_kodmi].apply(lambda col: col.apply(lambda t: standardize_text(t) if pd.notna(t) else t))

In [ ]:
# Chceme pocitat pocet spravenych vykonov, tak vytvor kombinaciu vykon-obdobie,
# aby sa vykon spravne zapocital viackrat, ak bol pocas jednej hospitalizacie
# robeny viackrat.
# Vytvor zoznam vykonov v tvare, ako ho pouzivame pre algoritmus. 
# Kedze uzs data neobsahuju datum vykonu, iba obdobie, v ktorom bola dodana starostlivost, pouzi namiesto dna xx
# Kedze uzs data neobsahuju lokalizaciu vykonu, pouzi kod Z "neznama lokalizacia"

_uzs['operacny_vykon'] = _uzs['kod_hlavneho_vykonu'] + '&Z&' + _uzs['obdobie'].str.replace('-', '') + 'xx'
_uzs['jednodnovy_vykon'] = _uzs['kod_jednodnoveho_vykonu'] + '&Z&' + _uzs['obdobie'].str.replace('-', '') + 'xx'

_uzs = _uzs.drop(columns=['kod_hlavneho_vykonu', 'kod_jednodnoveho_vykonu'])


### Zjednoť dáta v `uzs`

In [ ]:
# Zjednot celu tabulku tak, aby id_hp bolo unikatnym identifikatorom (kazdy riadok samostatne id_hp).

# Pomocná funkcia na jednoriadkový zápis odstránenia prvku z množiny
def _discard(s, e):
    s.discard(e)
    return s

# pomocná funkcia na jednoriadkový zápis odstránenia nan hodnôt
def _dropna(a):
    return a[pd.notnull(a)]

# pomocna funkcia na vlozenie prvku na zaciatok
def _insert_first(l, a):
    l.insert(0, a)
    return l

# Tabuľka `uzs` zoskupená podľa id_hp, na ktorom sa budú agregovať jednotlivé stĺpce
_grouped_uzs = _uzs.sort_values(by=['obdobie', 'datum_od', 'datum_do'], ascending=True, na_position='first').groupby(by=['kod_zp', 'id_hp'])

# Tabuľka, do ktorej sa budú ukladať výstupné zjednotenia
zjednotene_uzs = pd.DataFrame()

In [ ]:
# 'datum_od' vezmi najmensi
# 'datum_do' vezmi najvacsi
# 'obdobie_od' vezmi najmensie obdobie
# 'obdobie_do' vezmi najvacsie obdobie
# (Mozeme si dovolit pouzit first a last kedze je to zoradene, min a max su o dost pomalsie)

zjednotene_uzs['datum_od'] = _grouped_uzs['datum_od'].first()
zjednotene_uzs['datum_do'] = _grouped_uzs['datum_do'].last()
zjednotene_uzs['obdobie_od'] = _grouped_uzs['obdobie'].first()
zjednotene_uzs['obdobie_do'] = _grouped_uzs['obdobie'].last()

In [ ]:
# vykony: vytvor zoznam vykonanych vykonov a k tomu prisluchajuce typy vykonov

zjednotene_uzs['vykony'] = _grouped_uzs[['operacny_vykon', 'jednodnovy_vykon']].apply(lambda z: '~'.join(_dropna(z.values.flatten()))).replace('', pd.NA)
# typ_vykonu sa nenachadza v datach za 2024
# zjednotene_uzs['typ_vykonu'] = _grouped_uzs[['typ_vykonu']].apply(lambda z: '~'.join(_dropna(z.values.flatten()))).replace('', pd.NA)


In [ ]:
# diagnozy: 
#   posledna znama diagnoza pri prepusteni je hlavna diagnoza
#       ak nebola ziadna dianoza pri prepusteni, vezmi poslednu znamu diagnozu pri prijati 
#   vytvor mnozinu vsetkych diagnoz, bude obsahovat aj hlavnu diagnozu na zaciatku

zjednotene_uzs['posledna_dgn_prepustenie'] = _grouped_uzs['dgn_prepustenie'].last()
zjednotene_uzs['posledna_dgn_prijem'] = _grouped_uzs['dgn_prijem'].last()
zjednotene_uzs['hlavna_diagnoza'] = zjednotene_uzs['posledna_dgn_prepustenie'].fillna(zjednotene_uzs['posledna_dgn_prijem'])

zjednotene_uzs['diagnozy'] = _grouped_uzs[['dgn_prijem', 'dgn_prepustenie']].apply(lambda z: set(_dropna(z.values.flatten())))
zjednotene_uzs['diagnozy'] = zjednotene_uzs[['hlavna_diagnoza', 'diagnozy']].apply(lambda r: '~'.join(_insert_first(list(_discard(r['diagnozy'], r['hlavna_diagnoza'])), r['hlavna_diagnoza'])), axis=1)

In [ ]:
# 'typ_hospitalizacie', 'typ_starostlivosti', 'novorodenec', 'pzs_12' a 'pzs_12_odosielatel' vezmi prve
# 'pohyb_poistenca', 'id_poistenca' vezmi posledne (stavalo sa pri porodoch, ze jedno id_hp, malo dvoch poistencov, kedze najprv to bolo pisane na matku az neskor na dieta)

zjednotene_uzs['typ_hospitalizacie'] = _grouped_uzs['typ_hospitalizacie'].first()
zjednotene_uzs['typ_starostlivosti'] = _grouped_uzs['typ_starostlivosti'].first()
zjednotene_uzs['novorodenec'] = _grouped_uzs['novorodenec'].first()
zjednotene_uzs['pzs_12'] = _grouped_uzs['pzs_12'].first()
zjednotene_uzs['pzs_12_odosielatel'] = _grouped_uzs['pzs_12_odosielatel'].first()

zjednotene_uzs['posledny_pohyb_poistenca'] = _grouped_uzs['pohyb_poistenca'].last()
zjednotene_uzs['id_poistenca'] = _grouped_uzs['id_poistenca'].last()


In [ ]:
# Vypis statistiky

# napoveda k nazvom stlpcov v povodnom R kode:
# ID_POI_ZP = id_poistenca
# ID_HOSP_ZP = id_hp
# DAT_PRIJ = datum_od
# DAT_PREP = datum_do
# KOD_VYK_OPER = kod_operacneho_vykonu
# TYP_ZS = typ_hospitalizacie
# ins_name = kod_zp
# nemP01 = pzs_8
# id_hosp_nem = id_hp_pzs_8
# diag_uzs = dgn_prijem
# vykon_uzs = kod_vykonu_obdobie

print(len(zjednotene_uzs))
print(zjednotene_uzs.reset_index().nunique())

487137
kod_zp                           2
id_hp                       487137
datum_od                       453
datum_do                       366
obdobie_od                      24
obdobie_do                      12
vykony                       31266
posledna_dgn_prepustenie      6955
posledna_dgn_prijem           7156
hlavna_diagnoza               7133
diagnozy                     21787
typ_hospitalizacie               7
typ_starostlivosti               4
novorodenec                      3
pzs_12                        1845
pzs_12_odosielatel           10807
posledny_pohyb_poistenca       385
id_poistenca                301650
dtype: int64


## Priprava dat DRG

### Priprav data zo suboru 02_HP_UDAJE

In [ ]:
# Najdi relevantne tabulky
_drg_full = pd.concat([_data[i]['hp'] for i in _spracovat], ignore_index=True)

len(_drg_full)

362404

In [ ]:
# Vyhod nepotrebne stlpce (v tomto pripade ziadne) a duplikaty

# napoveda k nazvom stlpcov v povodnom R subore
# ID_HP_DZP = id_hp
# vek_zapis = vek_roky
# HMOTNOST = hmotnost
# UPV = upv
# KOD_DIAG_HL = hlavna_diagnoza
# KOD_DRG = drg
# DAT_OD = datum_od
# DAT_DO = datum_do
# KOD_PZS = pzs_6
# TYP_ZS = typ_starostlivosti
# ins_name = kod_zp
# ID_HP_PZS = id_hp_pzs

drg = _drg_full.drop_duplicates()

print(len(drg))
print(drg.nunique())


362404
kod_zp                               2
id_hp_pzs                        68719
id_hp                           362404
id_poistenca                    234073
pzs_6                               90
datum_od                         71156
datum_do                         45501
osetrovacia_doba                   144
dni_priepustka                      18
vek_dni                            366
vek_roky                           104
hmotnost                          1786
upv                                694
datum_narodenia                  33540
druh_prijatia                        9
dovod_prijatia                       7
dovod_prepustenia                   10
hlavna_diagnoza                   6428
lokalizacia_hlavnej_diagnozy         4
drg                               1113
erv                               9954
zlucene_hp                        3931
uhrada_hp                       107662
uhrada_pp                         9235
uhrada_imzs                      30361
uhrada_emzs       

In [ ]:
# VALIDATION chybajuce data

id_validacie = 'druh_prijatia_chybajuci'
invalid = drg[drg['druh_prijatia'].isna()]
register_validation(invalid, id_validacie)
id_validacie = 'dovod_prijatia_chybajuci'
invalid = drg[drg['dovod_prijatia'].isna()]
register_validation(invalid, id_validacie)
id_validacie = 'dovod_prepustenia_chybajuci'
invalid = drg[drg['dovod_prepustenia'].isna()]
register_validation(invalid, id_validacie)
id_validacie = 'drg_chybajuci'
invalid = drg[drg['drg'].isna()]
register_validation(invalid, id_validacie)
id_validacie = 'erv_chybajuci'
invalid = drg[drg['erv'].isna()]
register_validation(invalid, id_validacie)

In [ ]:
# VALIDATION zaporne uhrady

id_validacie = 'uhrada_zaporna_02'
invalid = drg[drg[UHRADY_COLUMNS].lt(0).any(axis=1)].copy()
register_validation(invalid, id_validacie)

### Priprav data zo suboru 03_HP_PREKLADY

In [ ]:
# vyber relevantne tabulky
preklady = pd.concat([_data[i]['preklady'] for i in _spracovat], ignore_index=True)

print(len(preklady))
print(preklady.nunique())

462047
kod_zp           2
id_hp       362401
pzs_12        1126
datum_od       466
datum_do       443
dtype: int64


In [ ]:
# Priprav na pripojenie k suboru 02_HP_UDAJE
# vytvor pzs_6 ako prvych 6 cislic z pzs_12
# Prirad prve pzs_12 na ktore bolo hp prijate a koresponduje s uvedenym pzs_6 

preklady['pzs_6'] = preklady['pzs_12'].str[:6]

_grouped_preklady = preklady.sort_values(by=['datum_od', 'datum_do'], ascending=True).groupby(by=['kod_zp', 'id_hp', 'pzs_6'])
drg = drg.merge(_grouped_preklady['pzs_12'].first(), on=['kod_zp', 'id_hp', 'pzs_6'], how='left')

print(len(drg))
print(drg.nunique())

362404
kod_zp                               2
id_hp_pzs                        68719
id_hp                           362404
id_poistenca                    234073
pzs_6                               90
datum_od                         71156
datum_do                         45501
osetrovacia_doba                   144
dni_priepustka                      18
vek_dni                            366
vek_roky                           104
hmotnost                          1786
upv                                694
datum_narodenia                  33540
druh_prijatia                        9
dovod_prijatia                       7
dovod_prepustenia                   10
hlavna_diagnoza                   6428
lokalizacia_hlavnej_diagnozy         4
drg                               1113
erv                               9954
zlucene_hp                        3931
uhrada_hp                       107662
uhrada_pp                         9235
uhrada_imzs                      30361
uhrada_emzs       

#### VALIDATION preklady_rozni_poskytovatelia

In [ ]:
# Exportuj preklady, ktore pre rovnaky id_hp maju zaznamenany preklad medzi roznymi poskytovatelmi

id_validacie = 'preklady_rozni_poskytovatelia'

_unique_pzs6 = preklady.groupby(['kod_zp', 'id_hp'])['pzs_6'].unique()
_unique_pzs6 = _unique_pzs6[_unique_pzs6.str.len() > 1]

invalid = drg.set_index(['kod_zp', 'id_hp']).loc[_unique_pzs6.index].reset_index()
invalid['unique_pzs6'] = _unique_pzs6.values

register_validation(invalid, id_validacie)

In [ ]:
# Exportuj zoznam hospitalizacii, ktore nie su zaznamenane v subore 03_PREKLADY

id_validacie = 'preklad_chybajuci'

invalid = drg[drg['pzs_12'].isna()]

register_validation(invalid, id_validacie)

### Priprav data zo suboru 04_HP_VDG

In [ ]:
# Najdi relevantne tabulky

vdg = pd.concat([_data[i]['vdg'] for i in _spracovat], ignore_index=True)

len(vdg)

688547

In [ ]:
# Vyhod duplicity

vdg = vdg.drop_duplicates()

print(len(vdg))
print(vdg.nunique())

688545
kod_zp          2
id_hp      236503
vdg          9063
lok_vdg         4
dtype: int64


In [ ]:
# Vytvor mnozinu vedlajsich diagnoz

vdg['vdg'] = vdg['vdg'].apply(lambda t: standardize_text(t) if pd.notna(t) else t)
_vedlajsie_diagnozy = vdg.groupby(['kod_zp', 'id_hp'])[['vdg', 'lok_vdg']].agg(lambda z: '~'.join([x for x in z if pd.notna(x)])).rename(columns={'vdg': 'vedlajsie_diagnozy', 'lok_vdg': 'lokalizacie_vedlajsich_diagnoz'})

In [ ]:
# Pripoj vedlajsie diagnozy zo suboru 04_HP_VDG k suboru 02_HP

drg = drg.merge(_vedlajsie_diagnozy, how='left', on=['kod_zp', 'id_hp'])

print(len(drg))

362404


### Priprav data zo suboru 05_HP_ZV

In [ ]:
# Vyber relevantne tabulky 

vykony = pd.concat([_data[i]['vykony'] for i in _spracovat], ignore_index=True)

In [ ]:
# Vytvor identifikator vykonu na zaklade jeho kodu, datumu vykonania a
# lokalizacie. Tiez na zaklade jeho kodu, roku a mesiaca vykonania na porovnanie
# so suborom 01_UZS_JZS.

vykony['kod_vykonu'] = vykony['kod_vykonu'].apply(standardize_text)
vykony.loc[:, ['kod_vykonu']] = vykony['kod_vykonu'] + '&' + vykony['lokalizacia_vykonu'].fillna('Z') + '&' + vykony['datum_vykonu'].dt.strftime(r'%Y%m%d').fillna('xxxxxxxx')

In [ ]:
# Vytvor zoznam vykonov a Pripoj vykony zo suboru 05_HP_ZV k suboru 02_HP

_zoznamy_vykonov = vykony.groupby(['kod_zp', 'id_hp'])['kod_vykonu'].apply(lambda z: '~'.join(z)).rename('vykony')

drg = drg.merge(_zoznamy_vykonov, how='left', on=['kod_zp', 'id_hp'])

print(len(drg))

362404


### Priprav data zo suboru 06_PPDRG

In [ ]:
# Vyber relevantne tabulky 

ppdrg = pd.concat([_data[i]['ppdrg'] for i in _spracovat], ignore_index=True)

In [ ]:
ppdrg.columns

Index(['kod_zp', 'id_hp', 'kod_drg_pp', 'cena_drg_pp'], dtype='object')

In [ ]:
# Validation: cena je zaporna

id_validacie = 'cena_pp_zaporna'

invalid = ppdrg[ppdrg['cena_drg_pp'].lt(0)].copy()
invalid['pzs_6'] = pd.NA
invalid['id_hp_pzs'] = pd.NA
invalid['id_poistenca'] = pd.NA

register_validation(invalid, id_validacie)

In [ ]:
# Pre kazde id_hp vytvor zoznam pripocitatelnych poloziek a ich cien

_pripocitatelne_polozky = ppdrg.groupby(['kod_zp', 'id_hp'])[['kod_drg_pp', 'cena_drg_pp']].agg({'kod_drg_pp': lambda z: '~'.join(z), 'cena_drg_pp': lambda z: '~'.join([str(x) for x in z])})

In [ ]:
drg = drg.merge(_pripocitatelne_polozky, how='left', on=['kod_zp', 'id_hp'])

print(len(drg))
print(drg.nunique())

362404
kod_zp                                 2
id_hp_pzs                          68719
id_hp                             362404
id_poistenca                      234073
pzs_6                                 90
datum_od                           71156
datum_do                           45501
osetrovacia_doba                     144
dni_priepustka                        18
vek_dni                              366
vek_roky                             104
hmotnost                            1786
upv                                  694
datum_narodenia                    33540
druh_prijatia                          9
dovod_prijatia                         7
dovod_prepustenia                     10
hlavna_diagnoza                     6428
lokalizacia_hlavnej_diagnozy           4
drg                                 1113
erv                                 9954
zlucene_hp                          3931
uhrada_hp                         107662
uhrada_pp                           9235
uhrada_im

## Doplnenie DRG dat z UZS_JZS

In [ ]:
# Pripoj data z 01_UZS_JZS k existujucim datam z 02_HP_UDAJE

final = drg.merge(zjednotene_uzs, how='left', on=['kod_zp', 'id_hp'], suffixes=['_02', '_01'])

print(len(final))

print(final.nunique())

362404
kod_zp                                 2
id_hp_pzs                          68719
id_hp                             362404
id_poistenca_02                   234073
pzs_6                                 90
datum_od_02                        71156
datum_do_02                        45501
osetrovacia_doba                     144
dni_priepustka                        18
vek_dni                              366
vek_roky                             104
hmotnost                            1786
upv                                  694
datum_narodenia                    33540
druh_prijatia                          9
dovod_prijatia                         7
dovod_prepustenia                     10
hlavna_diagnoza_02                  6428
lokalizacia_hlavnej_diagnozy           4
drg                                 1113
erv                                 9954
zlucene_hp                          3931
uhrada_hp                         107662
uhrada_pp                           9235
uhrada_im

###  Zjednot udaje zo suborov 01_UZS_JZS a 02_HP_UDAJE

Prioritu dostava subor 02_HP_UDAJE (`02`)

#### ID poistenca

In [ ]:
if int(_rok) > 2021:
    id_validacie = 'id_poistenca_nezhodny'

    final['id_poistenca'] = final['id_poistenca_02'].fillna(final['id_poistenca_01'])

    invalid = final[final['id_poistenca_02'] != final['id_poistenca_01']]
    
    register_validation(invalid, id_validacie)

    final = final.drop(columns=['id_poistenca_01', 'id_poistenca_02'])

####   Typ zdravotnej starostlivosti

In [ ]:
# Typ starostlivosti
id_validacie = 'typ_starostlivosti_nezhodny'

invalid = final[final['typ_starostlivosti_02'] != final['typ_starostlivosti_01']]

register_validation(invalid, id_validacie)

# Add separate validation for UH type in T02
id_validacie = 'typ_starostlivosti_UH_v_02'

invalid = final[final['typ_starostlivosti_02'] == 'UH']

register_validation(invalid, id_validacie)

# resolve invalid cases (Priority T02, 'UH')
final['typ_starostlivosti'] = final['typ_starostlivosti_02'].fillna(final['typ_starostlivosti_01']).fillna('UH')
final = final.drop(columns=['typ_starostlivosti_01', 'typ_starostlivosti_02'])

#### Kod poskytovatela pzs_12

In [ ]:
final['pzs_12'] = final['pzs_12_02'].fillna(final['pzs_12_01'])
final = final.drop(columns=['pzs_12_01', 'pzs_12_02'])


#### Datumy

In [ ]:
id_validacie = 'datumy_nezhodne'

invalid = final[final[['datum_od_01', 'datum_od_02']].notna().all(axis=1) & ((final['datum_od_02'] - final['datum_od_01']).dt.days != 0) | final[['datum_do_01', 'datum_do_02']].notna().all(axis=1) & ((final['datum_do_02'] - final['datum_do_01']).dt.days != 0)]

register_validation(invalid, id_validacie)

final['datum_od'] = final['datum_od_02'].fillna(final['datum_od_01'])
final['datum_do'] = final['datum_do_02'].fillna(final['datum_do_01'])

final = final.drop(columns=['datum_od_01', 'datum_do_01', 'datum_od_02', 'datum_do_02'])

#### Diagnozy

In [ ]:
# kod hlavnej diagnozy a drg zjednot na standardny tvar male pismena, cisla 
final[['hlavna_diagnoza_02', 'drg']] = final[['hlavna_diagnoza_02', 'drg']].apply(lambda col: col.apply(lambda t: standardize_text(t) if pd.notna(t) else t))

In [ ]:
id_validacie = 'hlavna_diagnoza_nezhodna'

invalid = final[final['hlavna_diagnoza_01'] != final['hlavna_diagnoza_02']]

register_validation(invalid, id_validacie)

id_validacie = 'hlavna_diagnoza_chybajuca_02'

invalid = final[final['hlavna_diagnoza_02'].isna()]

register_validation(invalid, id_validacie)

final['hlavna_diagnoza'] = final['hlavna_diagnoza_02'].fillna(final['hlavna_diagnoza_01'])

final = final.drop(columns=['hlavna_diagnoza_01', 'hlavna_diagnoza_02'])

In [ ]:
# Ako vedlajsie diagnozy pouzi vedlajsie diagnozy z 02 a ponechaj pre analyticke ucely diagnozy z 01

final = final.rename(columns={'diagnozy': 'diagnozy_z_01'})

In [ ]:
# Vytvor stlpec `diagnozy` obsahujuci hlavnu diagnozu na zaciatku

_spojka = np.where(final['vedlajsie_diagnozy'].notna(), '~', '')

final['diagnozy'] = final['hlavna_diagnoza'].fillna('') + _spojka + final['vedlajsie_diagnozy'].fillna('')

final = final.drop(columns=['hlavna_diagnoza', 'vedlajsie_diagnozy'])

#### Vykony

In [ ]:
# Ako vykony pouzi vykony z 02 a ponechaj pre analyticke ucely vykony z 01

final = final.rename(columns={'vykony_02': 'vykony', 'vykony_01': 'vykony_z_01'}) 

## Pripojenie dalsich hospitalizacnych pripadov z UZS_JZS

In [ ]:
# Vyber iba tie hp, ktoré sa nenachádzajú v drg dátach
# uz nebudeme potrebovat stlpec s hlavnou diagnozou

zjednotene_uzs = zjednotene_uzs.reset_index()
_doplnkove_uzs = zjednotene_uzs[~zjednotene_uzs['id_hp'].isin(final['id_hp'])]
_doplnkove_uzs = _doplnkove_uzs.drop(columns=['hlavna_diagnoza'])
_doplnkove_uzs[['diagnozy_z_01', 'vykony_z_01']] = _doplnkove_uzs[['diagnozy', 'vykony']]

len(_doplnkove_uzs)

124733

In [ ]:
final = pd.concat([final, _doplnkove_uzs]).reset_index()

print(len(final))
print(final.nunique())

487137
index                             407022
kod_zp                                 2
id_hp_pzs                          68719
id_hp                             487137
pzs_6                                 90
osetrovacia_doba                     144
dni_priepustka                        18
vek_dni                              366
vek_roky                             104
hmotnost                            1786
upv                                  694
datum_narodenia                    33540
druh_prijatia                          9
dovod_prijatia                         7
dovod_prepustenia                     10
lokalizacia_hlavnej_diagnozy           4
drg                                 1113
erv                                 9954
zlucene_hp                          3931
uhrada_hp                         107662
uhrada_pp                           9235
uhrada_imzs                        30361
uhrada_emzs                        41133
typ_kontraktu                          3
lokalizac

## finalna uprava vsetkych hospitalizacii

### Pripojenie dat zo suboru 09_UZS_POISTENCI

In [ ]:
poistenci = pd.concat([_data[i]['poistenci'] for i in _spracovat], ignore_index=True)

print(len(poistenci))
print(poistenci.nunique())

2777596
kod_zp                          2
id_poistenca              2664505
datum_narodenia             35788
pohlavie                        2
datum_poistenia_od          10941
datum_poistenia_do           1999
dovod_ukoncenia                22
kod_prechodneho_pobytu       2868
psc_prechodneho_pobytu       2102
kod_trvaleho_pobytu          2926
psc_trvaleho_pobytu         13807
dtype: int64


In [ ]:
# Vylúč záznamy, ktorým skončilo poistenie pred daným rokom

poistenci = poistenci[~(poistenci['datum_poistenia_do'].dt.year < int(_rok))]

# Vylúč záznamy, ktoré skončili v danom roku a prešli do inej poisťovne

poistenci = poistenci[~((poistenci['datum_poistenia_do'] < f'{_rok}-12-31') & poistenci['dovod_ukoncenia'].isin(['24','25','27']))]

# Vylúč záznamy, ktorým začína poistenie až v budúcom roku

poistenci = poistenci[~(poistenci['datum_poistenia_od'].dt.year > int(_rok))]

print(len(poistenci))
print(poistenci.nunique())

2493186
kod_zp                          2
id_poistenca              2465833
datum_narodenia             35097
pohlavie                        2
datum_poistenia_od          10803
datum_poistenia_do            924
dovod_ukoncenia                20
kod_prechodneho_pobytu       2861
psc_prechodneho_pobytu       2087
kod_trvaleho_pobytu          2926
psc_trvaleho_pobytu         13805
dtype: int64


In [ ]:
# Vylúč duplicity
poistenci = poistenci.drop_duplicates()

len(poistenci)

2493119

In [ ]:
# Skontroluj platný dôvod ukončenia

platne_dovody_ukoncenia = ['24', '25', '27', 'M', 'K', 'C', 'E', 'P', 'T', 'R', 'Z']

id_validacie = 'poistenci_dovod_ukoncenia_neplatny'

invalid = poistenci[poistenci['dovod_ukoncenia'].notna() & ~poistenci['dovod_ukoncenia'].isin(platne_dovody_ukoncenia)].copy()
invalid['pzs_6'] = pd.NA
invalid['id_hp'] = pd.NA
invalid['id_hp_pzs'] = pd.NA

register_validation(invalid, id_validacie)

In [ ]:
# Najdi poistencov, ktori maju viac ako jeden datum narodenia a vyhod ich

# Pozn. možno bude nutné spätne ponechať tzv. fiktivnych poistencov, su to id_poistenca, u ktorych sa moze objavit viacnasobny datum narodenia, pretoze sa pouziju pre viacerych pacientov, ktorych poistenie je nezname alebo utajene. Ak takí existujú, ich zoznam sa nachádza v súbore poznamky.txt pri pri ostatných súboroch od poisťovne.   

id_validacie = 'poistenci_viac_datumov_narodenia'

invalid = poistenci[poistenci.groupby('id_poistenca')['datum_narodenia'].transform('nunique') > 1]
invalid['pzs_6'] = pd.NA
invalid['id_hp'] = pd.NA
invalid['id_hp_pzs'] = pd.NA

register_validation(invalid, id_validacie)

print(len(poistenci))
print(poistenci.nunique())

2493119
kod_zp                          2
id_poistenca              2465833
datum_narodenia             35097
pohlavie                        2
datum_poistenia_od          10803
datum_poistenia_do            924
dovod_ukoncenia                20
kod_prechodneho_pobytu       2861
psc_prechodneho_pobytu       2087
kod_trvaleho_pobytu          2926
psc_trvaleho_pobytu         13805
dtype: int64


In [ ]:
# Vytvor kod pobytu a psc pobytu primarne z prechodneho pobytu

poistenci['kod_pobytu'] = poistenci['kod_prechodneho_pobytu'].fillna(poistenci['kod_trvaleho_pobytu'])
poistenci['psc_pobytu'] = poistenci['psc_prechodneho_pobytu'].fillna(poistenci['psc_trvaleho_pobytu'])

In [ ]:
# Vyber posledny zaznamenany pobyt
poistenci = poistenci.groupby(by=['kod_zp', 'id_poistenca', 'pohlavie', 'datum_narodenia'])[['psc_pobytu', 'kod_pobytu']].last().reset_index()

print(len(poistenci))
print(poistenci.nunique())

2465833
kod_zp                   2
id_poistenca       2465833
pohlavie                 2
datum_narodenia      35097
psc_pobytu            5699
kod_pobytu            2929
dtype: int64


In [ ]:
# Skontroluj zaznamenaný ZUJ kód a PSČ pobytu

poistenci['psc_pobytu'] = poistenci['psc_pobytu'].str.replace(' ', '')

id_validacie = 'poistenci_ZUJ_a_PSC_chybajuce'

invalid = poistenci[poistenci['kod_pobytu'].isna() & poistenci['psc_pobytu'].isna()].copy()
invalid['pzs_6'] = pd.NA
invalid['id_hp'] = pd.NA
invalid['id_hp_pzs'] = pd.NA
register_validation(invalid, id_validacie)

obce = load_zoznam_obci('Inputs/KÓDY_OBCE.xlsx')
psc = load_psc_2_zuj('Inputs/psc_na_zuj.csv')

id_validacie = "poistenci_ZUJ_nedohladane"

invalid = poistenci[poistenci['kod_pobytu'].notna() & ~poistenci['kod_pobytu'].isin(obce['ZUJ'])].copy()
invalid['pzs_6'] = pd.NA
invalid['id_hp'] = pd.NA
invalid['id_hp_pzs'] = pd.NA
register_validation(invalid, id_validacie)

id_validacie = "poistenci_PSC_nedohladane"

invalid = poistenci[poistenci['psc_pobytu'].notna() & ~poistenci['psc_pobytu'].isin(psc['psc'].unique())].copy()
invalid['pzs_6'] = pd.NA
invalid['id_hp'] = pd.NA
invalid['id_hp_pzs'] = pd.NA
register_validation(invalid, id_validacie)

id_validacie = "poistenci_pohlavie_chybajuce"

invalid = poistenci[poistenci['pohlavie'].isna()].copy()
invalid['pzs_6'] = pd.NA
invalid['id_hp'] = pd.NA
invalid['id_hp_pzs'] = pd.NA
register_validation(invalid, id_validacie)

In [ ]:
# Pre chýbajúci alebo neplatný ZUJ kód priraď ZUJ kód podľa PSČ (vyber náhodne jeden zo ZUJ kódov pokrytých daným PSČ proporčne k počtu obyvateľov v danom ZUJ kóde - predpočítané v prevodovom subore psc_na_zuj)

doplnenie_zuj = poistenci[poistenci['psc_pobytu'].isin(psc['psc'].unique()) & ~poistenci['kod_pobytu'].isin(obce['ZUJ'].unique())]

poistenci.loc[doplnenie_zuj.index, ['kod_pobytu']] = doplnenie_zuj.merge(psc.groupby('psc').agg({'ZUJ': list, 'ZUJ_pravdepodobnost': list}), left_on='psc_pobytu', right_index=True).apply(lambda r: random.choices(r['ZUJ'], r['ZUJ_pravdepodobnost'])[0], axis=1)

In [ ]:
# Vytvor exporty vyfiltrovaných poistencov

poistenci.to_csv(_paths['vystupy'] / f'poistenci_vyfiltrovani_{_rok}.csv', sep=';', index=False)

In [ ]:
# Pripoj data zo suboru 09_UZS_POISTENCI

final = final.merge(poistenci, how='left', on=['kod_zp', 'id_poistenca'], suffixes=['_02', '_09'])

In [ ]:
# Exportuj ID poistencov, ktore mame v datach o hospitalizaciach ale nie v datach o poistencoch

id_validacie = "id_poistenca_nedohladane"

invalid = final[~final['id_poistenca'].isin(poistenci['id_poistenca'])]

register_validation(invalid, id_validacie)

In [ ]:
# vyexportuj pripady, kde sa nezhoduju datumy narodenia, a datum z 02 nie je novorodenec (datum narodenia skor ako v decembri minuleho roka)
# zjednot datumy narodenia, primarny je z 09

if int(_rok) > 2021:
    
    id_validacie = 'datum_narodenia_nezhodny'

    invalid = final[final[['datum_narodenia_02', 'datum_narodenia_09']].notna().all(axis=1) & (final['datum_narodenia_02'] != final['datum_narodenia_09']) & (final['datum_narodenia_02'] < pd.Timestamp(year=int(_rok)-1, month=12, day=1))]

    register_validation(invalid, id_validacie)
    
    final['datum_narodenia'] = final['datum_narodenia_09'].fillna(final['datum_narodenia_02'])

    final = final.drop(columns=['datum_narodenia_02', 'datum_narodenia_09'])

### Kod poskytovatela pzs_8

In [ ]:
# Pridaj kod nemocnice podla prevodovnika na zaklade kodov oddeleni.

_pzs_12_prevodnik = load_pzs_12('Inputs/pzs_12_prevodnik.csv')

final = final.merge(_pzs_12_prevodnik, how='left', on='pzs_12')

len(final)

487137

In [ ]:
final['pzs_6'] = final['pzs_6_x'].fillna(final['pzs_6_y'])

final = final.drop(columns=['pzs_6_x', 'pzs_6_y'])

In [ ]:
# Exportuj kody oddeleni, ktorym nevieme priradit kod nemocnice

final[final['pzs_8'].isna()][['id_hp', 'pzs_6', 'pzs_8', 'pzs_12']].to_csv(_paths['validacia'] / 'pzs_12_nedohladane_v_prevodniku.csv', sep=';', index=False)

id_validacie = "pzs_12_nedohladane_v_datach_nczi"

invalid = final[final['pzs_8'].fillna('xxxxxxxx').str.endswith('xx')]

register_validation(invalid, id_validacie)

In [ ]:
# Ak sa nepripojil kod nemocnice, lebo hospitalizacia nema zaznam v subore
# 03_HP_PREKLADY, alebo je hospitalizacia z 01_UZS_JZS vytvor kod nemocnice 
# ako 'KOD_PZS + 0 + Y', kde Y sa zisti z polozky ID_HP_PZS ako (Z)RRYXXXXX. 
# Y by nikdy nemalo byt 0, ale nemocnice si ho tak niekedy udavaju, takze sprav zmenu na 1.

_y=final['id_hp_pzs'].str.extract('^Z?\d\d(\d)').replace('0', '1')

final['pzs_8'] = final['pzs_8'].fillna(final['pzs_6'] + '0' + _y[0])

### Vyradenie hospitalizacii nekonciacich vo vybranom roku

In [ ]:
id_validacie = 'konci_v_zlom_obdobi'

_konci_v_zlom_obdobi = ((final['datum_do'].notna()) & (final['datum_do'].dt.year != int(_rok)))

invalid = final[_konci_v_zlom_obdobi]

register_validation(invalid, id_validacie)

final = final.drop(final[_konci_v_zlom_obdobi].index)

len(final)

487137

### Dlzka osetrovacej doby

In [ ]:
# Napocitaj dlzku osetrovacej doby ako rozdiel v dnoch medzi datumom prepustenia a datumom prijatia. Pouzi v pripade, ze nebola dlzka osetrovacej doby reportovana.

def vypocitaj_dlzku_osetrovacej_doby(datum_prijatia, datum_prepustenia):
    if datum_prepustenia < datum_prijatia:
        return 1
    return max(1,(datum_prepustenia - datum_prijatia).days)


_vypocitana_dlzka_osetrovania = final.apply(lambda row: vypocitaj_dlzku_osetrovacej_doby(row['datum_od'], row['datum_do']), axis=1).astype('Int16')

In [ ]:
id_validacie = "osetrovacia_doba_0"

invalid = final[final['osetrovacia_doba'].notna() & (final['osetrovacia_doba'] == 0)]

register_validation(invalid, id_validacie)

final.loc[final['osetrovacia_doba'].notna() & (final['osetrovacia_doba'] == 0), 'osetrovacia_doba'] = pd.NA

In [ ]:
final.loc[:,'osetrovacia_doba'] = final['osetrovacia_doba'].fillna(_vypocitana_dlzka_osetrovania)
final.loc[final['typ_starostlivosti'].isin(['JZS', 'JZS2']), 'osetrovacia_doba'] = final.loc[final['typ_starostlivosti'].isin(['JZS', 'JZS2']), 'osetrovacia_doba'].fillna(1)

### Vek

In [ ]:
# Natipuj vek podla datumov hospitalizacii a narodenia a toho, co bolo napisane.
# Z datumov hospitalizacii sa najde minimalny a podla neho a datumu narodenia
# sa urci vek pri hospitalizacii. 

final['datum_prijatia'] = final[['datum_od', 'datum_do']].min(axis=1)

In [ ]:
# Vypocitaj vek presne zo zadanych datumov prijatia a narodenia

def vypocitaj_vek(datum_prijatia, datum_narodenia):
    if datum_prijatia < datum_narodenia:
        return 0, 1
    vek_roky = datum_prijatia.year - datum_narodenia.year - ((datum_prijatia.month, datum_prijatia.day) < (datum_narodenia.month, datum_narodenia.day))
    if vek_roky == 0:
        return vek_roky, (datum_prijatia - datum_narodenia).days
    return vek_roky, pd.NA

final[['vypocitany_vek_roky', 'vypocitany_vek_dni']] = final.apply(lambda row: vypocitaj_vek(row['datum_prijatia'], row['datum_narodenia']), axis=1, result_type='expand')

In [ ]:
# Pre pripady, ze je znamy datum narodenia ale neznamy datum prijatia, mozeme odhadnut vek ako jednoduchy rozdiel medzi aktualnym rokom a rokom narodenia

final['odhadnuty_vek_roky'] = final.apply(lambda row: int(_rok) - row['datum_narodenia'].year, axis=1)

In [ ]:
# Nakoniec ak je vyplneny vek v dnoch alebo hmotnost, vek je 0 rokov.

final.loc[(final['vek_dni'] > 0) | (final['hmotnost'] > 0), ['odhadnuty_vek_0']] = 0

In [ ]:
# Prioritu ma zaznamenany vek v rokoch, doplna sa vypocitanym alebo odhadnutym vekom

final['vek_roky'] = final['vek_roky'].fillna(final['odhadnuty_vek_0'])
final['vek_roky'] = final['vek_roky'].fillna(final['vypocitany_vek_roky'])
final['vek_dni'] = final['vek_dni'].fillna(final['vypocitany_vek_dni'])
final['vek_roky'] = final['vek_roky'].fillna(final['odhadnuty_vek_roky'])


final = final.drop(
    columns=['datum_prijatia', 'vypocitany_vek_roky', 'vypocitany_vek_dni', 'odhadnuty_vek_roky', 'odhadnuty_vek_0']
    )

In [ ]:
# Exportuj zaznamy, ktore maju nestandardne hodnoty
# Vysoky vek
id_validacie="vek_vysoky"
invalid=final[final['vek_roky'] > 109]
register_validation(invalid, id_validacie)

# Novorodenci bez hmotnosti (dopln dohodnutu hodnotu 2500)
id_validacie="novorodenec_bez_hmotnosti"
invalid=final[(final['vek_roky'] == 0) & (final['vek_dni'] <= 28) & (final['hmotnost'].isna())]
register_validation(invalid, id_validacie)

final.loc[(final['vek_roky'] == 0) & ((final['vek_dni'] <= 28) | final['vek_dni'].isna()) & (final['hmotnost'].isna()), 'hmotnost'] = 2500

# Chybajuci vek (aj napriek carovaniu so vsetkym moznym)
id_validacie="vek_chybajuci"
invalid=final[final['vek_roky'].isna()]
register_validation(invalid, id_validacie)

In [ ]:
# TODO premysliet, ci nie je vhodnejsie ako najpravdivejsi vek povazovat ten z tabulky 09.

In [ ]:
final['vek_roky'] = pd.to_numeric(final['vek_roky'], errors='coerce').astype('Int16')
final['vek_dni'] = pd.to_numeric(final['vek_dni'], errors='coerce').astype('Int16')

### Vylucenie hospitalizacii, ktore boli zlucene kvoli DRG

In [ ]:
# Zo stlpca s ciastkovymi hospitalizaciami odstran medzery (2021 ich nema, ale
# radsej pre isototu to treba spravit) a ak je ako oddelovac pouzita bodka
# (mala by byt pouzivana len ciarka, ak nie je treba to vratit poistovni)
# zmen ju na ciarku

final['zlucene_hp'] = final['zlucene_hp'].str.replace(' ', "").str.replace('.', ',')

In [ ]:
# Vyber riadky, ktore maju nieco v ciastkovej hospitalizacii
_zlucene = final[final['zlucene_hp'].notna()][['pzs_8', 'zlucene_hp']].drop_duplicates()

In [ ]:
# Rozdel ciastkove hospitalizacie samostatne do riadkov

_zlucene = _zlucene.set_index(['pzs_8']).apply(lambda col: col.str.split(',').explode()).reset_index()

In [ ]:
# Vytvor jedinecne id pre ciastkove hospitalizacie
_zlucene['id_hp_pzs_pzs_8'] = _zlucene['zlucene_hp'] + '_' + _zlucene['pzs_8']

In [ ]:
# Najdi id hospitalizacii prisluchajucich jedinecnym id

final['id_hp_pzs_pzs_8'] = final['id_hp_pzs'] + '_' + final['pzs_8']

_id_na_vylucenie = final[final['id_hp_pzs_pzs_8'].isin(_zlucene['id_hp_pzs_pzs_8'])]['id_hp']

In [ ]:
# Exportuj ciastkove hospitalizacie
final.loc[final['id_hp'].isin(_id_na_vylucenie),['id_hp', 'id_hp_pzs']].to_csv(_paths['validacia'] / 'zlucene_hospitalizacie.csv', sep=';', index=False)

In [ ]:
# Vyhod ciastkove hospitalizacie

final = final.drop(final[final['id_hp'].isin(_id_na_vylucenie)].index)

final = final.drop(columns=['id_hp_pzs_pzs_8'])

len(final)

479468

### Reporting chybajucich dat

In [ ]:
final = final.replace('', np.nan)

id_validacie = 'id_poistenca_chybajuce'
invalid = final[final['id_poistenca'].isna()]
register_validation(invalid, id_validacie)

id_validacie = 'typ_starostlivosti_chybajuci'
invalid = final[final['typ_starostlivosti'].isna()]
register_validation(invalid, id_validacie)

id_validacie = 'pzs_12_chybajuce'
invalid = final[final['pzs_12'].isna()]
register_validation(invalid, id_validacie)

id_validacie = 'datumy_chybajuce'
invalid = final[(final['datum_od'].isna()) | (final['datum_do'].isna() & (~final['typ_starostlivosti'].isin(['JZS', 'JZS2'])))]
register_validation(invalid, id_validacie)

id_validacie = 'hlavna_diagnoza_chybajuca'
invalid = final[final['diagnozy'].str.startswith('~', na=False)]
register_validation(invalid, id_validacie)

id_validacie = 'diagnozy_chybajuce'
invalid = final[final['diagnozy'].isna()]
register_validation(invalid, id_validacie)

## Upratanie finalnych dat

In [ ]:
# Popresuvaj stlpce a vypis statistiky

final = final[COLUMNS_VS]

print(len(final))
print(final.nunique())

479468
id_hp                             479468
vek_roky                             104
vek_dni                              366
hmotnost                            1779
upv                                  692
diagnozy                          169350
lokalizacia_hlavnej_diagnozy           4
lokalizacie_vedlajsich_diagnoz      7047
vykony                            245726
drg                                 1113
erv                                 9711
typ_starostlivosti                     4
typ_hospitalizacie                     7
druh_prijatia                          9
dovod_prijatia                         7
dovod_prepustenia                     10
posledna_dgn_prijem                 7145
posledna_dgn_prepustenie            6940
posledny_pohyb_poistenca             384
diagnozy_z_01                      21614
vykony_z_01                        31107
novorodenec                            3
obdobie_od                            24
obdobie_do                            12
datum_od 

In [ ]:
# Exportuj do csv

final.to_csv(_paths['vystupy'] / f'osn_vsetka_starostlivost_{_rok}_{"_".join(_spracovat)}.csv', sep=';', index=False)

## Priprava dat pre 274

In [ ]:
final_274 = pd.DataFrame(columns=COLUMNS_274.values())
final_274['dgn_prijem'] = final['posledna_dgn_prijem']
final_274['dgn_prepustenie'] = final['posledna_dgn_prepustenie']
final_274['pohyb_poistenca'] = final['posledny_pohyb_poistenca']
final_274['novorodenec'] = final['novorodenec']
final_274['pzs_12_odosielatel'] = final['pzs_12_odosielatel']
final_274['id_poistenca'] = final['id_poistenca']
final_274['pohlavie_poistenca'] = final['pohlavie']
# final_274['typ_vykonu'] = final['typ_vykonu'].str.replace('~', '@')

_vykony_list = final['vykony'].str.split('~')
_kody_vykonov = _vykony_list.apply(lambda z: [x.split('&')[0] for x in z] if isinstance(z, list) else z)
_lokalizacie_vykonov = _vykony_list.apply(lambda z: [x.split('&')[1] for x in z] if isinstance(z, list) else z)
_datumy_vykonov = _vykony_list.apply(lambda z: [x.split('&')[2] for x in z] if isinstance(z, list) else z)

final_274['kod_hlavneho_vykonu'] = _kody_vykonov.str[0]
final_274['typ_hospitalizacie'] = final['typ_hospitalizacie']
final_274['datum_prijatia'] = final['datum_od'].dt.date
final_274['datum_prepustenia'] = final['datum_do'].dt.date
final_274['id_hp'] = final['id_hp']
final_274['upv'] = final['upv']
final_274['kody_vykonov'] = _kody_vykonov.str.join('@')
final_274['lokalizacie_vykonov'] = _lokalizacie_vykonov.str.join('@')
final_274['datumy_vykonov'] = _datumy_vykonov.str.join('@')
final_274['datum_narodenia'] = final['datum_narodenia']
final_274['druh_prijatia'] = final['druh_prijatia']
final_274['dovod_prijatia'] = final['dovod_prijatia']
final_274['vek_dni'] = final['vek_dni']
final_274['vek_roky'] = final['vek_roky']
final_274['hmotnost'] = final['hmotnost']
final_274['druh_prepustenia'] = final['dovod_prepustenia']
final_274['kod_hlavnej_diagnozy'] = final['diagnozy'].str.split('~').str[0]
final_274['lokalizacia_hlavnej_diagnozy'] = final['lokalizacia_hlavnej_diagnozy']
final_274['kody_vedlajsich_diagnoz'] = final['diagnozy'].str.split('~').apply(lambda z: '@'.join(z[1:]) if len(z) > 1 else pd.NA)
final_274['lokalizacie_vedlajsich_diagnoz'] = final['lokalizacie_vedlajsich_diagnoz'].str.replace('~', '@')
final_274['dlzka_osetrovacej_doby'] = final['osetrovacia_doba']
final_274['dni_priepustka'] = final['dni_priepustka']
final_274['drg'] = final['drg']
final_274['erv'] = final['erv']
final_274['kod_drg_pp'] = final['kod_drg_pp'].str.replace('~', '@')
final_274['cena_drg_pp'] = final['cena_drg_pp'].str.replace('~', '@')
final_274['cas_prijatia'] = final['datum_od'].dt.time
final_274['cas_prepustenia'] = final['datum_do'].dt.time
final_274['zlucovane_hp'] = final['zlucene_hp']
final_274['pzs_ico'] = final['pzs_ico']
final_274['id_hp_pzs'] = final['id_hp_pzs']

pzs_ico_centralne_riadenie = ['37957937', '00606715', '00165549', '17336007', '00610470', '00610381', '17335825', '00165336', '00607231', '31813861', '00606707', '00365327', '00227811', '35971126', '36513458', '36644331', '36603350', '36601284']
final_274 = final_274.query('pzs_ico.isin(@pzs_ico_centralne_riadenie)')

In [ ]:
# Exportuj do csv

final_274.to_csv(_paths['vystupy'] / f'f274_{"_".join(_spracovat)}_osn_vsetka_starostlivost_f274_{_rok}.csv', sep='|', index=False)

## Priprava dat pre algoritmus

In [ ]:
# Pridaj stlpec s odbornostami (aktualne prazdny) a vyber iba chcene stlpce

_verzia_algoritmu = 'v2024.2'

if _verzia_algoritmu == 'v2024.1':
    _for_algorithm = final[['id_hp', 'vek_roky', 'vek_dni', 'hmotnost', 'upv', 'diagnozy', 'vykony', 'drg']]
    _for_algorithm.insert(7, "odbornosti", pd.NA)
            
    _doplneny_vek_dni = _for_algorithm[_for_algorithm['vek_roky'] >= 1]['vek_dni'].fillna(0)
    _for_algorithm.loc[_doplneny_vek_dni.index,['vek_dni']] = _doplneny_vek_dni
elif _verzia_algoritmu == 'v2024.2':
    _for_algorithm = final[['id_hp', 'vek_roky', 'hmotnost', 'upv', 'diagnozy', 'vykony', 'drg']]


In [ ]:
# Dopln dalsie numericke hodnoty na 0

_doplnena_hmotnost = _for_algorithm[~(_for_algorithm['vek_roky'] == 0)]['hmotnost'].fillna(0)
_for_algorithm.loc[_doplnena_hmotnost.index,['hmotnost']] = _doplnena_hmotnost

_for_algorithm.loc[:, ['upv']] = _for_algorithm['upv'].fillna(0)

In [ ]:
# Vyexportuj do csv

print(len(_for_algorithm))

print(_for_algorithm.nunique())

_for_algorithm.to_csv(_paths['vystupy'] / f'osn_vsetka_starostlivost_{_rok}_pre_algoritmus_{_verzia_algoritmu}.csv', sep=';', header=False, index=False)


479468
id_hp       479468
vek_roky       104
hmotnost      1779
upv            693
diagnozy    169350
vykony      245726
drg           1113
dtype: int64


# Validácia s tabuľkou 13

In [ ]:
# Najdi relevantne tabuľky
sumar = pd.concat([_data[k]['sumar'] for k in _spracovat], ignore_index=True)
sumar['pocet'] = sumar['pocet'].astype('Int32')

len(sumar)

685

In [ ]:
# Vytvor sumar z final v rovnakom tvare, ako je vykazovany poistovanmi v tabulke 13 a porovnaj

_final_sumar = final.groupby(by=['kod_zp', 'pzs_6', 'typ_starostlivosti'])['id_hp'].count().astype('Int32').rename('pocet')

sumar = sumar.merge(_final_sumar, on=['kod_zp', 'pzs_6', 'typ_starostlivosti'], how='outer', suffixes=['_vykazany', '_spocitany'])

sumar['rozdiel_absolutny'] = sumar['pocet_vykazany'].fillna(0) - sumar['pocet_spocitany'].fillna(0)
sumar['rozdiel_relativny'] = (sumar['rozdiel_absolutny'] / sumar[['pocet_spocitany', 'pocet_vykazany']].max(axis=1)).abs()
sumar = sumar.set_index(['kod_zp', 'pzs_6', 'typ_starostlivosti'])

In [ ]:
sumar

pocet_vykazany  pocet_spocitany  \
kod_zp pzs_6  typ_starostlivosti                                    
24     N19681 DRG                            838              838   
              JZS                             62               59   
              JZS2                             1                1   
       N21149 DRG                           1399             1381   
              JZS2                           140              140   
...                                          ...              ...   
27     P97554 DRG                            388              376   
              JZS                              4                4   
              UH                              32               32   
       P99327 JZS                             56               56   
       P99795 UH                               7                7   

                                  rozdiel_absolutny  rozdiel_relativny  
kod_zp pzs_6  typ_starostlivosti                                        
24     N19681 DRG                                 0                0.0  
              JZS                                 3           0.048387  
              JZS2                                0                0.0  
       N21149 DRG                                18           0.012866  
              JZS2                                0                0.0  
...                                             ...                ...  
27     P97554 DRG                                12           0.030928  
              JZS                                 0                0.0  
              UH                                  0                0.0  
       P99327 JZS                                 0                0.0  
       P99795 UH                                  0                0.0  

[722 rows x 4 columns]

In [ ]:
sumar.to_csv(_paths['validacia'] / 'sumar_porovnanie.csv', sep=';', index=True)

id_validacie = 'sumar_vysoke_rozdiely'
invalid = sumar[sumar['rozdiel_relativny'] > 0.05].reset_index().copy()
invalid['id_hp'] = pd.NA
invalid['id_hp_pzs'] = pd.NA
invalid['id_poistenca'] = pd.NA
register_validation(invalid, id_validacie)

In [ ]:
sumar.groupby(['typ_starostlivosti', 'kod_zp'])['rozdiel_absolutny'].sum().reset_index()

,typ_starostlivosti,kod_zp,rozdiel_absolutny
0,DRG,24,4146
1,DRG,27,1564
2,JZS,24,315
3,JZS,27,282
4,JZS2,24,-6
5,UH,24,-12021
6,UH,27,0


In [ ]:
# Ako by sa zmenili pocty, pokial by sa pripocitali aj vylucene HP 

sumar.groupby(['typ_starostlivosti', 'kod_zp'])['rozdiel_absolutny'].sum() - drg[drg['id_hp'].isin(_id_na_vylucenie)].groupby(['typ_starostlivosti', 'kod_zp'])['id_hp'].count().fillna(0)

typ_starostlivosti  kod_zp
DRG                 24       -1957.0
                    27           0.0
JZS                 24          <NA>
                    27          <NA>
JZS2                24          -8.0
UH                  24          <NA>
                    27          <NA>
dtype: Float64

# Rozdeľ validačné výstupy podľa poisťovní

In [ ]:
for _kod in _spracovat:
    with pd.ExcelWriter(_paths['validacia'] / f'validacia_{_rok}_{_kod}.xlsx', engine='openpyxl') as writer:
        VALIDATIONS[_kod].to_excel(writer, sheet_name='validacny_protokol', index=False)
        VALIDATIONS[_kod][['nazov_validacie', 'zavaznost']].value_counts().to_frame('count').to_excel(writer, sheet_name=f'statistiky_validacie', index=True)
        if _kod in sumar.index:
            sumar.loc[_kod].to_excel(writer, sheet_name=f'sumar_porovnanie', index=True)